# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [18]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [4]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")

# Display sample data
city_data_df.head(5)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Georgetown,MY,5.4112,100.3354,87.73,62,20,8.05,few clouds
1,1,Westport,US,41.1415,-73.3579,39.56,64,0,9.22,clear sky
2,2,Nantucket,US,41.2835,-70.0995,39.34,79,100,5.75,overcast clouds
3,3,Torbay,CA,47.6666,-52.7314,30.97,86,100,20.71,heavy snow
4,4,Arraial Do Cabo,BR,-22.9661,-42.0278,76.96,94,0,10.36,clear sky


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [5]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [6]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[
    (city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)
]

# Display sample data
preferred_cities_df.head(5)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
4,4,Arraial Do Cabo,BR,-22.9661,-42.0278,76.96,94,0,10.36,clear sky
7,7,Galle,LK,6.0367,80.2170,80.76,72,100,6.17,overcast clouds
10,10,Tabou,CI,4.4230,-7.3528,79.93,87,42,4.88,scattered clouds
14,14,Oriximina,BR,-1.7656,-55.8661,73.67,95,92,2.89,overcast clouds
15,15,Urupes,BR,-21.2017,-49.2900,78.13,90,100,4.88,overcast clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [7]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities.head(5)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
4,4,Arraial Do Cabo,BR,-22.9661,-42.0278,76.96,94,0,10.36,clear sky
7,7,Galle,LK,6.0367,80.2170,80.76,72,100,6.17,overcast clouds
10,10,Tabou,CI,4.4230,-7.3528,79.93,87,42,4.88,scattered clouds
14,14,Oriximina,BR,-1.7656,-55.8661,73.67,95,92,2.89,overcast clouds
15,15,Urupes,BR,-21.2017,-49.2900,78.13,90,100,4.88,overcast clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [8]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities.copy()

## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [9]:
# Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(5)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Hotel Name
4,4,Arraial Do Cabo,BR,-22.9661,-42.0278,76.96,94,0,10.36,clear sky,
7,7,Galle,LK,6.0367,80.2170,80.76,72,100,6.17,overcast clouds,
10,10,Tabou,CI,4.4230,-7.3528,79.93,87,42,4.88,scattered clouds,
14,14,Oriximina,BR,-1.7656,-55.8661,73.67,95,92,2.89,overcast clouds,
15,15,Urupes,BR,-21.2017,-49.2900,78.13,90,100,4.88,overcast clouds,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [10]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [11]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head(10)

Starting hotel search
Arraial Do Cabo - nearest hotel: No hotel found
Galle - nearest hotel: Jungle Hostel Villa
Tabou - nearest hotel: hôtel le rochet
Oriximina - nearest hotel: No hotel found
Urupes - nearest hotel: No hotel found
Victoria - nearest hotel: 四季酒店 Four Seasons Hotel
Quelimane - nearest hotel: Mabassa
Taoudenni - nearest hotel: No hotel found
Sachin - nearest hotel: No hotel found
Ambilobe - nearest hotel: Hôtel National
Takoradi - nearest hotel: Hillcrest
Mahebourg - nearest hotel: Grand Bel Air
Saint-Philippe - nearest hotel: Le Baril
Altagracia De Orituco - nearest hotel: Hotel El Canastro
Hithadhoo - nearest hotel: Pebbles Inn
Sao Joao Da Barra - nearest hotel: Pousada Mediterrâneo
Mahajanga - nearest hotel: Vahiny Hotel
Mangan - nearest hotel: No hotel found
Rudbar - nearest hotel: No hotel found
Sao Filipe - nearest hotel: Bela Vista
Sirsilla - nearest hotel: No hotel found
Trincomalee - nearest hotel: Silver Star Inn
Micomeseng - nearest hotel: Hotel Victor
Buchan

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Hotel Name
4,4,Arraial Do Cabo,BR,-22.9661,-42.0278,76.96,94,0,10.36,clear sky,No hotel found
7,7,Galle,LK,6.0367,80.2170,80.76,72,100,6.17,overcast clouds,Jungle Hostel Villa
10,10,Tabou,CI,4.4230,-7.3528,79.93,87,42,4.88,scattered clouds,hôtel le rochet
14,14,Oriximina,BR,-1.7656,-55.8661,73.67,95,92,2.89,overcast clouds,No hotel found
15,15,Urupes,BR,-21.2017,-49.2900,78.13,90,100,4.88,overcast clouds,No hotel found
16,16,Victoria,HK,22.2855,114.1577,78.85,59,0,3.00,clear sky,四季酒店 Four Seasons Hotel
17,17,Quelimane,MZ,-17.8786,36.8883,75.15,94,75,21.99,light rain,Mabassa
18,18,Taoudenni,ML,22.6783,-3.9836,75.99,11,10,11.56,clear sky,No hotel found
20,20,Sachin,IN,21.0833,72.8833,78.69,44,0,3.44,haze,No hotel found
22,22,Ambilobe,MG,-13.2000,49.0500,75.38,90,98,2.26,overcast clouds,Hôtel National


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [12]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != "No hotel found"]

# Display sample data
clean_hotel_df.head(5)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Hotel Name
7,7,Galle,LK,6.0367,80.2170,80.76,72,100,6.17,overcast clouds,Jungle Hostel Villa
10,10,Tabou,CI,4.4230,-7.3528,79.93,87,42,4.88,scattered clouds,hôtel le rochet
16,16,Victoria,HK,22.2855,114.1577,78.85,59,0,3.00,clear sky,四季酒店 Four Seasons Hotel
17,17,Quelimane,MZ,-17.8786,36.8883,75.15,94,75,21.99,light rain,Mabassa
22,22,Ambilobe,MG,-13.2000,49.0500,75.38,90,98,2.26,overcast clouds,Hôtel National


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [13]:
# Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [20]:
# Configure the map plot
hotel_map = clean_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["City", "Hotel Name", "Country", "Description"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Description)